In [53]:
import pandas as pd
import pickle
import numpy as np
import sys
sys.path.append("../../")
from src.soporte_transformers import (
    load_models
)

In [54]:
df = pd.read_csv("../../datos/06_rent_outliers_gestionados.csv")
df.sample()

,size,bathrooms,municipality,district,distancia_centro,price,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,...,floor_primero,floor_quinto,floor_segundo,floor_septimo,floor_sexto,floor_sotano,floor_tercero,hasLift_desconocido,hasLift_no tiene ascensor,hasLift_tiene ascensor
214,-0.059259,0.0,-1.038726,-2.397725,-0.953234,450.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Creamos un dato nuevo
- Para probar nuestras predicciones

In [55]:
# Datos de una nueva casa para predicción
new_house = pd.DataFrame({
    'propertyType': ['flat'],  # Nueva categoría no vista
    'size': [85],
    'rooms': ["2 habitaciones"],
    'bathrooms': ["1 aseo"],
    'floor': ["primero"],
    'municipality': ['Madrid'],
    'district': ["Puente de Vallecas"],
    'hasLift' : 'tiene ascensor',
    'distancia_centro' : 'Entre 1 y 5 km'
})

df_new = pd.DataFrame(new_house)
df_new

,propertyType,size,rooms,bathrooms,floor,municipality,district,hasLift,distancia_centro
0,flat,85,2 habitaciones,1 aseo,primero,Madrid,Puente de Vallecas,tiene ascensor,Entre 1 y 5 km


In [56]:
target_encoder, one_hot_encoder, robust_scaler, modelo_xgb =  load_models()

In [57]:
df_pred = df_new.copy()
df_pred

,propertyType,size,rooms,bathrooms,floor,municipality,district,hasLift,distancia_centro
0,flat,85,2 habitaciones,1 aseo,primero,Madrid,Puente de Vallecas,tiene ascensor,Entre 1 y 5 km


In [58]:
one_hot_encoder

OneHotEncoder(dtype='float')

# Separar las columnas por el tipo de encoding que necesitan

In [59]:
cols_target = ["bathrooms","municipality","district","distancia_centro"]
cols_onehot = ["propertyType","rooms","floor","hasLift"]

### Hacer one_hot_encoder

In [60]:
test = one_hot_encoder.transform(df_pred[cols_onehot])
# Obtenemos los nombres de las columnas del codificador
column_names = one_hot_encoder.get_feature_names_out(cols_onehot)
# Convertimos a un DataFrame
onehot_df = pd.DataFrame(test.toarray(), columns=column_names)
onehot_df

,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,propertyType_penthouse,propertyType_studio,rooms_1 habitacion,rooms_2 habitaciones,rooms_3 habitaciones,rooms_4 habitaciones,...,floor_primero,floor_quinto,floor_segundo,floor_septimo,floor_sexto,floor_sotano,floor_tercero,hasLift_desconocido,hasLift_no tiene ascensor,hasLift_tiene ascensor
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Hacer target_encoder

In [61]:
df_pred["precio"] = np.nan
df_pred = target_encoder.transform(df_pred)
df_pred

,propertyType,size,rooms,bathrooms,floor,municipality,district,hasLift,distancia_centro,precio
0,flat,85,2 habitaciones,677.756303,primero,705.594108,684.0437,tiene ascensor,701.110187,NaN


### Eliminar las columnas de onehot y unir el df que habíamos generado previamente

In [62]:
df_pred.drop(columns= cols_onehot,inplace=True)
df_pred = pd.concat([df_pred, onehot_df], axis=1)
df_pred

,size,bathrooms,municipality,district,distancia_centro,precio,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,...,floor_primero,floor_quinto,floor_segundo,floor_septimo,floor_sexto,floor_sotano,floor_tercero,hasLift_desconocido,hasLift_no tiene ascensor,hasLift_tiene ascensor
0,85,677.756303,705.594108,684.0437,701.110187,NaN,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Feature- Scaling

In [63]:
cols_escalar = ["size","bathrooms","municipality","district","distancia_centro"]


In [64]:
df_pred[cols_escalar] = robust_scaler.transform(df_pred[cols_escalar])

In [65]:
df_pred

,size,bathrooms,municipality,district,distancia_centro,precio,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,...,floor_primero,floor_quinto,floor_segundo,floor_septimo,floor_sexto,floor_sotano,floor_tercero,hasLift_desconocido,hasLift_no tiene ascensor,hasLift_tiene ascensor
0,0.681481,0.0,0.619147,0.15483,0.020011,NaN,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [66]:
df_pred.drop(columns="precio",inplace=True)
modelo_xgb.predict(df_pred)

array([718.37396], dtype=float32)

# Funciona!
